
## 1. Giới thiệu và Cài đặt môi trường
**Giải thích:**
Bước đầu tiên là nạp các thư viện cần thiết cho việc xử lý dữ liệu lớn (Spark), xử lý mảng (Pandas, Numpy) và trực quan hóa dữ liệu (Matplotlib, Seaborn).
Chúng ta cũng sẽ khởi tạo một `SparkSession`, đây là điểm bắt đầu của mọi ứng dụng Spark.


In [ ]:

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnan, year, month, dayofmonth, hour, udf
from pyspark.sql.types import DoubleType, FloatType
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.clustering import KMeans
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("TaxiNYC_Analysis") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print("Spark Session đã được khởi tạo thành công!")



**Kết quả:**
Môi trường đã sẵn sàng. Spark Session đã được khởi tạo và chúng ta có thể bắt đầu làm việc với dữ liệu.



## 2. Nạp dữ liệu
**Giải thích:**
Chúng ta sẽ đọc file dữ liệu `taxi_data.csv` vào một Spark DataFrame. Tham số `inferSchema=True` giúp Spark tự động nhận diện kiểu dữ liệu của các cột, và `header=True` để sử dụng dòng đầu tiên làm tên cột.
Sau đó, chúng ta sẽ hiển thị cấu trúc (schema) và 5 dòng đầu tiên của dữ liệu để có cái nhìn tổng quan.


In [ ]:

# Đọc dữ liệu từ file CSV
df = spark.read.csv("taxi_data.csv", header=True, inferSchema=True)

# Hiển thị cấu trúc dữ liệu
df.printSchema()

# Hiển thị 5 dòng đầu tiên
df.show(5)



**Kết quả:**
Dữ liệu đã được nạp thành công. Lệnh `printSchema` cho thấy các cột như `tpep_pickup_datetime`, `fare_amount` đã được nhận diện. `df.show(5)` cho thấy dữ liệu mẫu, giúp ta hình dung cấu trúc bảng.



## 3. Kiểm tra dữ liệu trống
**Giải thích:**
Dữ liệu thực tế thường bị thiếu sót. Bước này sẽ kiểm tra xem có ô nào chứa giá trị `null` hoặc `NaN` không.
Chúng ta sẽ đếm số lượng giá trị thiếu trong mỗi cột. Đối với các cột số thực, chúng ta kiểm tra cả `NaN`.


In [ ]:

# Hàm tạo biểu thức kiểm tra null/nan tùy theo kiểu dữ liệu
def count_nulls(c, dtype):
    # Nếu là số thực (Double/Float), kiểm tra cả NULL và NaN
    if dtype in ('DoubleType', 'FloatType', 'double', 'float'):
        return count(when(col(c).isNull() | isnan(c), c)).alias(c)
    # Các kiểu khác chỉ kiểm tra NULL
    else:
        return count(when(col(c).isNull(), c)).alias(c)

# Tạo danh sách biểu thức cho tất cả các cột
aggs = [count_nulls(c, t) for c, t in df.dtypes]

# Thực hiện kiểm tra
df.select(aggs).show()



**Kết quả:**
Bảng kết quả hiển thị số lượng giá trị null cho từng cột. Nếu tất cả là 0, dữ liệu sạch. Nếu có giá trị, ta có thể cần phải xử lý (điền thêm hoặc xóa bỏ).



## 4. Kiểm tra dữ liệu ngoại lệ và Làm sạch dữ liệu
**Giải thích:**
Theo yêu cầu, dữ liệu Taxi NYC chỉ nên nằm trong giới hạn địa lý của New York.
- Vĩ độ (Latitude): 40.4774° N đến 40.9176° N
- Kinh độ (Longitude): -74.2591° W đến -73.7004° W
Chúng ta sẽ lọc bỏ các dòng dữ liệu nằm ngoài phạm vi này. Ngoài ra, chúng ta cũng lọc bỏ các chuyến đi có `passenger_count` <= 0 hoặc `fare_amount` <= 0 (dữ liệu lỗi).


In [ ]:

# Định nghĩa giới hạn địa lý
lat_min, lat_max = 40.4774, 40.9176
long_min, long_max = -74.2591, -73.7004

# Lọc dữ liệu hợp lệ
df_clean = df.filter(
    (col("pickup_latitude").between(lat_min, lat_max)) &
    (col("pickup_longitude").between(long_min, long_max)) &
    (col("dropoff_latitude").between(lat_min, lat_max)) &
    (col("dropoff_longitude").between(long_min, long_max)) &
    (col("passenger_count") > 0) &
    (col("fare_amount") > 0)
)

print(f"Số dòng dữ liệu gốc: {df.count()}")
print(f"Số dòng dữ liệu sau khi làm sạch: {df_clean.count()}")



**Kết quả:**
Chúng ta đã loại bỏ các bản ghi có tọa độ không hợp lệ hoặc số tiền/số hành khách không hợp lý. Số lượng dòng dữ liệu sau khi lọc được in ra để so sánh sự thay đổi.



## 5. Trực quan hóa dữ liệu
**Giải thích:**
Để hiểu rõ hơn về dữ liệu, chúng ta sẽ vẽ biểu đồ. Do dữ liệu Spark rất lớn, để vẽ đồ thị bằng Matplotlib/Seaborn, chúng ta thường lấy mẫu (sample) hoặc chuyển đổi một phần dữ liệu đã tổng hợp sang Pandas.
Ở đây, chúng ta sẽ vẽ biểu đồ phân tán (scatter plot) của các điểm đón khách (Pickup locations).


In [ ]:

# Lấy mẫu 10000 điểm để vẽ biểu đồ cho nhanh
pdf_sample = df_clean.sample(False, 0.1, seed=42).limit(10000).toPandas()

plt.figure(figsize=(10, 8))
sns.scatterplot(x="pickup_longitude", y="pickup_latitude", data=pdf_sample, s=5, alpha=0.5)
plt.title("Phân bố vị trí đón khách (Pickup Locations)")
plt.xlabel("Kinh độ")
plt.ylabel("Vĩ độ")
plt.grid(True)
plt.show()



**Kết quả:**
Biểu đồ hiển thị mật độ các điểm đón khách. Các khu vực tập trung dày đặc điểm đại diện cho trung tâm thành phố hoặc các điểm nóng giao thông.



## 6. Tương quan dữ liệu
**Giải thích:**
Chúng ta muốn biết liệu quãng đường (`trip_distance`), số hành khách (`passenger_count`) có ảnh hưởng đến tổng số tiền (`total_amount`) hay không.
Chúng ta sẽ sử dụng ma trận tương quan (Correlation Matrix).


In [ ]:

# Chọn các cột số để tính tương quan
num_cols = ['passenger_count', 'trip_distance', 'fare_amount', 'tip_amount', 'total_amount']
# Chuyển sang Pandas để tính correlation matrix (hiệu quả với số lượng cột ít)
corr_data = df_clean.select(num_cols).toPandas().corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_data, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Ma trận tương quan giữa các thuộc tính")
plt.show()



**Kết quả:**
Biểu đồ nhiệt (Heatmap) cho thấy hệ số tương quan.
- Tương quan càng gần 1: Tỷ lệ thuận mạnh (ví dụ: `trip_distance` và `total_amount`).
- Tương quan gần 0: Ít liên quan (ví dụ: `passenger_count` và `total_amount`).



## 7. Phân cụm dữ liệu (K-Means)
**Giải thích:**
Chúng ta sẽ gom nhóm các chuyến đi dựa trên vị trí đón khách (`pickup_latitude`, `pickup_longitude`). Điều này giúp xác định các "khu vực" (clusters) hoạt động chính của taxi.
Chúng ta sử dụng thuật toán K-Means của Spark MLlib.


In [ ]:

# Chuẩn bị dữ liệu đầu vào cho model (Vector features)
assembler = VectorAssembler(inputCols=["pickup_latitude", "pickup_longitude"], outputCol="features")
data_cluster = assembler.transform(df_clean)

# Khởi tạo K-Means với k=5 cụm
kmeans = KMeans().setK(5).setSeed(1)
model_km = kmeans.fit(data_cluster)

# Dự đoán cụm cho dữ liệu
predictions = model_km.transform(data_cluster)

# Hiển thị tâm các cụm (Cluster Centers)
centers = model_km.clusterCenters()
print("Tọa độ tâm các cụm:")
for center in centers:
    print(center)



**Kết quả:**
Thuật toán đã tìm ra 5 tọa độ trung tâm đại diện cho 5 khu vực chính. Các chuyến đi sẽ được gán nhãn thuộc về cụm nào gần nhất.



## 8. Dự đoán số tiền thanh toán (Hồi quy tuyến tính)
**Giải thích:**
Chúng ta sẽ xây dựng mô hình máy học để dự đoán `total_amount` dựa trên các yếu tố như `trip_distance`, `passenger_count`, `pickup_longitude`, `pickup_latitude`.
Quy trình:
1. Tạo vector đặc trưng (`features`).
2. Chia dữ liệu thành tập Train (80%) và Test (20%).
3. Huấn luyện mô hình Linear Regression.
4. Đánh giá mô hình trên tập Test.


In [ ]:

# 1. Tạo vector đặc trưng
feature_cols = ['trip_distance', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
assembler_lr = VectorAssembler(inputCols=feature_cols, outputCol="features")
data_lr = assembler_lr.transform(df_clean)

# 2. Chia tập Train/Test
train_data, test_data = data_lr.randomSplit([0.8, 0.2], seed=42)

# 3. Huấn luyện mô hình
lr = LinearRegression(featuresCol="features", labelCol="total_amount")
lr_model = lr.fit(train_data)

# 4. Dự đoán trên tập Test
lr_predictions = lr_model.transform(test_data)

# Hiển thị kết quả dự đoán so với thực tế
lr_predictions.select("features", "total_amount", "prediction").show(5)



**Kết quả:**
Bảng trên hiển thị giá trị thực (`total_amount`) và giá trị dự đoán (`prediction`). Chúng ta có thể thấy độ lệch giữa dự đoán và thực tế.



### Đánh giá độ chính xác của mô hình
**Giải thích:**
Sử dụng RMSE (Root Mean Squared Error) và R2 để đánh giá.
- RMSE càng nhỏ càng tốt.
- R2 càng gần 1 càng tốt (biểu thị mức độ mô hình giải thích được sự biến thiên của dữ liệu).


In [ ]:

evaluator = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(lr_predictions)

evaluator_r2 = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(lr_predictions)

print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R2 Score: {r2}")



**Kết quả:**
Kết quả RMSE và R2 cho biết hiệu suất của mô hình. Nếu R2 cao (>0.7), mô hình khá tốt. Nếu thấp, có thể cần thêm đặc trưng hoặc dùng mô hình phức tạp hơn (như Random Forest).



## 9. Phân tích thời điểm có nhiều chuyến đi
**Giải thích:**
Thay vì dùng mô hình dự đoán phức tạp, chúng ta sẽ phân tích thống kê để xem giờ nào trong ngày có số lượng chuyến đi nhiều nhất (Giờ cao điểm).
Chúng ta sẽ trích xuất giờ (`hour`) từ cột `tpep_pickup_datetime` và đếm số lượng chuyến đi theo giờ.


In [ ]:

# Trích xuất giờ từ thời gian đón
df_time = df_clean.withColumn("pickup_hour", hour(col("tpep_pickup_datetime")))

# Nhóm theo giờ và đếm số lượng
hourly_counts = df_time.groupBy("pickup_hour").count().orderBy("pickup_hour").toPandas()

# Vẽ biểu đồ
plt.figure(figsize=(10, 6))
sns.barplot(x="pickup_hour", y="count", data=hourly_counts, palette="viridis")
plt.title("Số lượng chuyến đi theo giờ trong ngày")
plt.xlabel("Giờ trong ngày (0-23)")
plt.ylabel("Số lượng chuyến đi")
plt.show()



**Kết quả:**
Biểu đồ cột hiển thị số lượng chuyến đi từng giờ. Cột cao nhất là thời điểm có nhiều chuyến đi nhất (thường là giờ tan tầm hoặc buổi tối). Cột thấp nhất là thời điểm ít khách nhất (thường là rạng sáng).
